# Project: Sentiment Analysis

## Profile
Name: Putu Widyantara Artanta Wibawa

Dicoding: https://www.dicoding.com/users/putu_waw



## Import Library

We will need to install `google_play_scraper` for scrapping and `Sastrawi` for stopwords and stemming.

In [1]:
!pip install -q google_play_scraper Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 16.0 MB/s eta 0:00:00


Import necessary library

In [2]:
import math
import re
from collections import Counter
from io import StringIO

import requests
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px

from sklearn.model_selection import train_test_split
from tensorflow import keras
from google_play_scraper import reviews_all, reviews, Sort
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

Create requirements file.

In [3]:
!pip freeze > requirements.txt

## Text Preprocessing

In [4]:
df = pd.read_csv('dataset.csv')

In [5]:
df

,content,score
0,"Senangnya belanja dishopee,tapi yang kesalnya ...",5
1,"Lemot,sering gangguan,layar sering ngeblank.. ...",5
2,"Aplikasi nya bagus,kalau mau belanja online po...",5
3,Pengelaman aplikasi shopee nya bekerja dengan ...,5
4,Salah satu Aplikasi terfavv aplikasi belanja y...,5
...,...,...
29995,"Hai aku pengguna shopee dari 2017, dan sekali ...",5
29996,Belanja di shopee adalah aflikasi yg sangat me...,5
29997,"Saya suka menggunakan shopee, saya sdh lama se...",5
29998,Kemarin saya beli hp di shopiee kecewa banget ...,5


Text preprocessing including normalizing, removing slangs, and stopwords.

In [6]:
def normalize_text(text: str) -> str:
    text = text.lower()  # case folding
    text = re.sub(r"[^a-zA-Z\s-]", "", text)  # keep only alphabet
    text = re.sub(r"\s+", " ", text)  # remove extra space in middle
    text = text.strip()  # remove extra space beginnning and end
    return text

In [7]:
def fix_slangwords(text):
    slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "plisss": "tolong", "bgttt": "banget", "indo": "indonesia", "bgtt": "banget", "ad": "ada", "rv": "redvelvet", "plis": "tolong", "pls": "tolong", "cr": "sumber", "cod": "bayar ditempat", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin"}

    words = text.split()
    fixed_words = []

    for word in words:
        if word in slangwords:
            fixed_words.append(slangwords[word])
        else:
            fixed_words.append(word)

    fixed_text = " ".join(fixed_words)
    return fixed_text

In [8]:
# stop words
stopword_factory = StopWordRemoverFactory()
stopword_removal = stopword_factory.create_stop_word_remover()


def remove_stopword(text: str) -> str:
    return stopword_removal.remove(text)

In [9]:
def preprocessing_text(text: str) -> str:
    text = normalize_text(text)
    text = fix_slangwords(text)
    text = remove_stopword(text)
    # text = stemming_text(text)
    return text

In [10]:
df["content"] = df["content"].apply(preprocessing_text)

In [11]:
df

,content,score
0,senangnya belanja dishopeetapi kesalnya belanj...,5
1,lemotsering gangguanlayar sering ngeblank pada...,5
2,aplikasi nya baguskalau mau belanja online pok...,5
3,pengelaman aplikasi shopee nya bekerja baik be...,5
4,salah satu aplikasi terfavv aplikasi belanja a...,5
...,...,...
29995,hai aku pengguna shopee dan sekali kecewa kmri...,5
29996,belanja shopee aflikasi sangat mendukung bisni...,5
29997,suka menggunakan shopee sdh lama sekali menggu...,5
29998,kemarin beli hp shopiee kecewa banget hp nya b...,5


## Labelling

In [12]:
def download_data(url: str) -> pd.DataFrame:
    response = requests.get(url)
    data = response.text
    data_io = StringIO(data)
    df = pd.read_csv(data_io, sep="\t")
    return df

Download negative lexicon

In [13]:
neg_df = download_data('https://raw.githubusercontent.com/fajri91/InSet/master/negative.tsv')
neg_df.head()

,word,weight
0,putus tali gantung,-2
1,gelebah,-2
2,gobar hati,-2
3,tersentuh (perasaan),-1
4,isak,-5


Download positive lexicon

In [14]:
pos_df = download_data('https://raw.githubusercontent.com/fajri91/InSet/master/positive.tsv')
pos_df.head()

,word,weight
0,hai,3
1,merekam,2
2,ekstensif,3
3,paripurna,1
4,detail,2


In [15]:
# Create dictionaries for fast lookup of word polarity
negative_dict = dict(zip(neg_df['word'], neg_df['weight']))
positive_dict = dict(zip(pos_df['word'], pos_df['weight']))

In [16]:
# calculate polarity
def calculate_polarity(text, positive_dict, negative_dict):
    words = text.split()
    polarity = 0
    for word in words:
        polarity += positive_dict.get(word, 0)
        polarity += negative_dict.get(word, 0)
    return polarity

In [17]:
df["polarity"] = df["content"].apply(
    lambda x: calculate_polarity(x, positive_dict, negative_dict)
)

In [18]:
df = df[["content", "polarity"]]

In [19]:
df.polarity.describe()

,polarity
count,30000.000000
mean,0.399933
std,10.079648
min,-62.000000
25%,-5.000000
50%,1.000000
75%,6.000000
max,65.000000


In [20]:
# assign label according polarity
def assign_label(polarity):
    # positive
    if polarity > -15:
        return 2
    # negative
    elif polarity < -16:
        return 0
    else:
        return 1

In [21]:
df["label"] = df["polarity"].apply(assign_label)

In [22]:
df

,content,polarity,label
0,senangnya belanja dishopeetapi kesalnya belanj...,-27,0
1,lemotsering gangguanlayar sering ngeblank pada...,-5,2
2,aplikasi nya baguskalau mau belanja online pok...,-18,0
3,pengelaman aplikasi shopee nya bekerja baik be...,16,2
4,salah satu aplikasi terfavv aplikasi belanja a...,7,2
...,...,...,...
29995,hai aku pengguna shopee dan sekali kecewa kmri...,5,2
29996,belanja shopee aflikasi sangat mendukung bisni...,-14,2
29997,suka menggunakan shopee sdh lama sekali menggu...,3,2
29998,kemarin beli hp shopiee kecewa banget hp nya b...,3,2


In [23]:
df = df[df["content"].apply(len) > 10]

## Data Visualization

### Class Distribution

In [24]:
def viz_label_distribution(df):
    label_counts = df["label"].value_counts().reset_index()
    label_counts["label"] = label_counts["label"].map(
        {0: "negative", 1: "neutral", 2: "positive"}
    )
    label_counts.columns = ["label", "count"]

    fig = px.pie(
        label_counts, values="count", names="label", title="Class Distribution"
    )
    return fig


viz_label_distribution(df)

### Length Distribution

In [25]:
def viz_length_distribution(df):
    df = df.copy()
    df["content_length"] = df["content"].apply(len)

    bins = list(range(0, max(df["content_length"]) + 25, 25))
    df["length_bin"] = pd.cut(df["content_length"], bins=bins)

    viz_df = df["length_bin"].value_counts().sort_index().reset_index()
    viz_df.columns = ["length_bin", "count"]
    viz_df["length_bin"] = viz_df["length_bin"].astype(str)

    fig = px.bar(
        viz_df,
        x="length_bin",
        y="count",
        title="Content Length Distribution",
        labels={"length_bin": "Content Length (Characters)", "count": "Count"},
    )
    return fig


viz_length_distribution(df)

### Top Words

In [26]:
def viz_top_n_words(df, n=10):
    all_text = " ".join(df["content"]).split()
    word_counts = Counter(all_text)
    word_counts_df = pd.DataFrame(word_counts.items(), columns=["word", "count"])

    top_words_df = (
        word_counts_df.sort_values(by="count", ascending=False).head(n).iloc[::-1]
    )
    fig = px.bar(
        top_words_df,
        x="count",
        y="word",
        orientation="h",
        title="Most Used Words",
        labels={"count": "Word Count", "word": "Words"},
        color="count",
    )

    return fig


viz_top_n_words(df)

## Modeling
We will use 3 schema for modeling:
1. LSTM, with training and testing: 90:10
2. DNN, with training and testing: 85:15
3. GRU, with training and testing: 80:20

In [27]:
X = df["content"].values
y = df["label"].values

# change to categorical
y = tf.keras.utils.to_categorical(y)

# tokenizer using top 5000 words
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)

# pad the text so it have same length
X_seq = tokenizer.texts_to_sequences(X)
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_seq)

In [28]:
# early stopping
class EarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get("val_accuracy") >= 0.93 and logs.get("accuracy") >= 0.93:
            print(f"\nTraining and validation accuracy already reach 0.93, stopping.")
            self.model.stop_training = True

# decay learning rate overtime
def scheduler(epoch, lr):
    return lr * math.exp(-0.1)


early_stopping = EarlyStopping()
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

### Model LSTM, Training Testing 90:10

In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X_padded, y, test_size=0.1, random_state=42
)

model_lstm = tf.keras.models.Sequential(
    [
        tf.keras.layers.Embedding(input_dim=5000, output_dim=128),
        tf.keras.layers.LSTM(
            64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5
        ),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)

model_lstm.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

history = model_lstm.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=1024,
    validation_split=0.2,
    callbacks=[early_stopping, lr_scheduler],
)

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 179ms/step - accuracy: 0.7209 - loss: 0.9106 - val_accuracy: 0.9283 - val_loss: 0.3685 - learning_rate: 9.0484e-04
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 13s 158ms/step - accuracy: 0.9285 - loss: 0.3869 - val_accuracy: 0.9283 - val_loss: 0.2764 - learning_rate: 8.1873e-04
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 218ms/step - accuracy: 0.9327 - loss: 0.3237 - val_accuracy: 0.9283 - val_loss: 0.2509 - learning_rate: 7.4082e-04
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.9348 - loss: 0.2734 - val_accuracy: 0.9283 - val_loss: 0.2395 - learning_rate: 6.7032e-04
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 155ms/step - accuracy: 0.9367 - loss: 0.2425 - val_accuracy: 0.9283 - val_loss: 0.2298 - learning_rate: 6.0653e-04
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 180ms/step - accuracy: 0.9334 - loss: 0.2369 - val_accuracy: 0.9283 - val_loss: 0.2225 - learning_rate: 5.4881e-04
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 155ms/step - accuracy

In [30]:
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy_lstm*100:.2f}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.9359 - loss: 0.2103
Test Accuracy: 93.70


### Model DNN, Training Testing 85:15

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X_padded, y, test_size=0.15, random_state=42
)

model_dnn = tf.keras.models.Sequential(
    [
        tf.keras.layers.Embedding(input_dim=5000, output_dim=128),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)

model_dnn.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

history = model_dnn.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping, lr_scheduler],
)

Epoch 1/50
319/319 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9243 - loss: 0.3705 - val_accuracy: 0.9278 - val_loss: 0.2593 - learning_rate: 9.0484e-04
Epoch 2/50
319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9341 - loss: 0.2622 - val_accuracy: 0.9278 - val_loss: 0.2361 - learning_rate: 8.1873e-04
Epoch 3/50
316/319 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9362 - loss: 0.2030
Training and validation accuracy already reach 0.93, stopping.
319/319 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9362 - loss: 0.2028 - val_accuracy: 0.9443 - val_loss: 0.1703 - learning_rate: 7.4082e-04


In [32]:
loss_dnn, accuracy_dnn = model_dnn.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy_dnn*100:.2f}")

141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9428 - loss: 0.1706
Test Accuracy: 94.66


### Model GRU, Training Testing 80:20

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    X_padded, y, test_size=0.2, random_state=42
)
model_gru = tf.keras.models.Sequential(
    [
        tf.keras.layers.Embedding(input_dim=5000, output_dim=128),
        tf.keras.layers.GRU(64, return_sequences=False),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)

model_gru.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

history = model_gru.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=1024,
    validation_split=0.2,
    callbacks=[early_stopping, lr_scheduler],
)

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.7397 - loss: 0.9892 - val_accuracy: 0.9276 - val_loss: 0.2920 - learning_rate: 9.0484e-04
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9305 - loss: 0.3397 - val_accuracy: 0.9276 - val_loss: 0.2579 - learning_rate: 8.1873e-04
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9350 - loss: 0.2701 - val_accuracy: 0.9276 - val_loss: 0.2460 - learning_rate: 7.4082e-04
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9335 - loss: 0.2491 - val_accuracy: 0.9276 - val_loss: 0.2335 - learning_rate: 6.7032e-04
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9337 - loss: 0.2329 - val_accuracy: 0.9276 - val_loss: 0.2248 - learning_rate: 6.0653e-04
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9349 - loss: 0.2082 - val_accuracy: 0.9276 - val_loss: 0.2148 - learning_rate: 5.4881e-04
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9355 

In [34]:
loss_gru, accuracy_gru = model_gru.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy_gru*100:.2f}")

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9348 - loss: 0.1995
Test Accuracy: 93.99


## Inference

In [35]:
model_list = [model_lstm, model_dnn, model_gru]
model_list_name = ["LSTM", "DNN", "GRU"]
choosen_model = np.argmax([accuracy_lstm, accuracy_dnn, accuracy_gru])
best_model = model_list[choosen_model]
print(f"Using model: {model_list_name[choosen_model]}")

Using model: DNN


Type the string on new data and run this cell to do inference.

In [36]:
new_data = "belanja disini emang bagus dan terpercaya, top deh"  # @param {"type":"string"}
new_data = [preprocessing_text(new_data)]

new_seq = tokenizer.texts_to_sequences(new_data)
new_padded = tf.keras.preprocessing.sequence.pad_sequences(
    new_seq, maxlen=X_padded.shape[1]
)
predictions = best_model.predict(new_padded)
predicted_class = np.argmax(predictions, axis=1)

class_names = {
    0: "negative",
    1: "neutral",
    2: "positive"
}
pred_prob = predictions[0][predicted_class[0]]
result = class_names[predicted_class[0]]
print(f"Result: {result}")
print(f"Probability: {pred_prob}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
Result: positive
Probability: 0.9921326041221619
